In [1]:
# Import Dependencies
import requests
import json
import pandas as pd
import time
import numpy as np
from config import yelp_key

In [2]:
# API Base URL
base_url = "https://api.yelp.com/v3/businesses/search?"

In [3]:
# API Headers
headers = {
    "accept" : "application/json",
    "Authorization" : "Bearer " + yelp_key 
}

In [4]:
# Create List of Categories (manually located)
categories = ['beaches','hanggliding','horsebackriding',
             'hiking','hot_air_balloons',
              'paddleboarding','parasailing',
              'sailing','snorkeling','ziplining', 
              'spas','hotsprings','massage','bedbreakfast','hotels','resorts',
              'skiresorts','tours', 'transportation','restaurants','nightlife'
             ]

len(categories)

21

In [5]:
# Import Lat/Long Database
csv_path = "../Resources/CitiesWGeolocation.csv"
locations_df = pd.read_csv(csv_path)
locations_df.head()

,Unnamed: 0,City,State/Province,Country,Latitude,Longitude
0,0,Lexington,Kentucky,United States,38.046407,-84.497039
1,1,San Diego,California,United States,32.717420,-117.162773
2,2,Cook Islands​,NaN,Cook Islands​,-19.996972,-157.785871
3,3,Park City,Utah,United States,40.646092,-111.497996
4,4,Newcastle Upon Tyne,England,United Kingdom,54.973847,-1.613157


In [33]:
# Normalize capitalization
locations_df = locations_df.rename(columns = {"Latitude": "latitude","Longitude":"longitude"})

In [6]:
# Gather Lists of Latitudes and Longitudes
latitudes_list = []
longitudes_list = []
for i in range(locations_df.shape[0]):
    latitudes_list.append(locations_df.loc[i].at["latitude"])
    longitudes_list.append(locations_df.loc[i].at["longitude"])
print(latitudes_list)
print(longitudes_list)

[38.0464066, 32.7174202, -19.99697155, 40.646092100000004, 54.97384739999999, -27.468968199999996, -28.0402165, 34.244058700000004, -36.718804999999996, 49.5041747, 42.64873625, 37.2395367, -32.728465, 17.223472100000002, 25.265347100000003, 50.67108245, 21.721746, 36.508976000000004, 32.0564572, -16.484598300000002, -33.928992, -19.1421421, -33.934444, -28.648333299999997, -45.0321923, 33.77217945, -6.166490799999999, -20.2759451, 45.437190799999996, 19.70318225, -4.6574977, -28.002373100000003, 44.4654236, 31.6258257, 37.9374939, 49.8879177, 43.10656029999999, 30.2711224, 15.2214956, 43.4832523, -33.953177600000004, 26.9154576, 13.1500331, 20.169626800000003, 51.53882410000001, 60.39430550000001, 24.578720999999998, -34.61341495, 34.862942600000004, 18.1850507, 12.51756625, -35.14181285, -34.4175, -33.8611665, -16.9206657, 44.958452799999996, -26.6544338, -34.427808299999995, 51.08668970000001, 44.279621, -37.814217600000006, -31.955896399999997, 51.453802200000005, 44.841225, 27.175

In [17]:
# Gather List of URLs
url_list = []
lat_lon_list = []
cat_list = []
for i in range(len(latitudes_list)):
        for x in range(len(categories)):
            latitude = latitudes_list[i]
            longitude = longitudes_list[i]
            category = categories[x]
            # Limit radius to appx 15 miles from the coordinates
            complete_url = base_url + "latitude=" + str(latitude) + "&longitude=" + str(longitude) + "&radius=40000&categories=" + category + "&sort_by=rating"
            url_list.append(complete_url)
            lat_lon_list.append([latitude,longitude])
            cat_list.append(category)
# Confirm list of API calls is less than 5,000 daily limit
print(len(url_list))
url_list

4620


['https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=beaches&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=hanggliding&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=horsebackriding&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=hiking&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=hot_air_balloons&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=paddleboarding&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=parasailing&sort_by=rating',
 'https://api.yelp.com/v

In [20]:
# Create API Calls (separate the blank list to a new cell so we can do numerous calls if needed)
data_list=[]

In [22]:
record_count = 1
set_count = 1
url_errors = []


for i in range(len(url_list)):
    if (i%100 == 0 and i >=100):
        set_count += 1
        record_count = 0
        
    print("processing Record %s of Set %s | %s" %(record_count, set_count,url_list[i]))    
    
    record_count += 1

    response = requests.get(url_list[i],headers=headers)
    data = response.json()
    try:
        # Calculate Average Rating and Review Count
        rating_sum = 0
        review_sum = 0
        # setting the range with an initial number so it can be changed if an error pops up midway through
        for x in range(len(data["businesses"])):
            rating_sum += data["businesses"][x]["rating"]
            review_sum += data["businesses"][x]["review_count"]
        rating_avg = rating_sum / len(data["businesses"])
        review_avg = review_sum / len(data["businesses"])
    
        # Add the desired data to a dictionary
        data_dict = {
            "latitude" : lat_lon_list[i][0] ,
            "longitude": lat_lon_list[i][1],
            "category" : cat_list[i],
            "results_total" : len(data["businesses"]),
            "avg_rating" : rating_avg,
            "avg_review_count" : review_avg
        }
        # Append the dictionary to a list
        data_list.append(data_dict)
        time.sleep(1)
    except:
        print("********URL error: Skipping...********")
        print(data)
        url_errors.append({"url":url_list[i],"latitude":data["region"]["center"]["latitude"],
                           "longitude":data["region"]["center"]["longitude"]})
        pass

processing Record 1 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=beaches&sort_by=rating
processing Record 2 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=hanggliding&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -84.4970393, 'latitude': 38.0464066}}}
processing Record 3 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=horsebackriding&sort_by=rating
processing Record 4 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=hiking&sort_by=rating
processing Record 5 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=hot_air_balloons&sort_by=rating
********URL error: Skippin

processing Record 43 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=-19.99697155&longitude=-157.78587140620786&radius=40000&categories=beaches&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -157.78587140620786, 'latitude': -19.99697155}}}
processing Record 44 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=-19.99697155&longitude=-157.78587140620786&radius=40000&categories=hanggliding&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -157.78587140620786, 'latitude': -19.99697155}}}
processing Record 45 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=-19.99697155&longitude=-157.78587140620786&radius=40000&categories=horsebackriding&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -157.78587140620786, 'latitude': -19.99697155}}}
process

processing Record 72 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=40.646092100000004&longitude=-111.4979963&radius=40000&categories=snorkeling&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -111.4979963, 'latitude': 40.646092100000004}}}
processing Record 73 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=40.646092100000004&longitude=-111.4979963&radius=40000&categories=ziplining&sort_by=rating
processing Record 74 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=40.646092100000004&longitude=-111.4979963&radius=40000&categories=spas&sort_by=rating
processing Record 75 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=40.646092100000004&longitude=-111.4979963&radius=40000&categories=hotsprings&sort_by=rating
processing Record 76 of Set 1 | https://api.yelp.com/v3/businesses/search?latitude=40.646092100000004&longitude=-111.4979963&radius=40000&categories=massa

processing Record 13 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-27.468968199999996&longitude=153.02349909999998&radius=40000&categories=snorkeling&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 153.02349909999998, 'latitude': -27.468968199999996}}}
processing Record 14 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-27.468968199999996&longitude=153.02349909999998&radius=40000&categories=ziplining&sort_by=rating
processing Record 15 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-27.468968199999996&longitude=153.02349909999998&radius=40000&categories=spas&sort_by=rating
processing Record 16 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=-27.468968199999996&longitude=153.02349909999998&radius=40000&categories=hotsprings&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 153.0234990999

processing Record 52 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=34.244058700000004&longitude=-116.91018000000001&radius=40000&categories=paddleboarding&sort_by=rating
processing Record 53 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=34.244058700000004&longitude=-116.91018000000001&radius=40000&categories=parasailing&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -116.91018000000001, 'latitude': 34.244058700000004}}}
processing Record 54 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=34.244058700000004&longitude=-116.91018000000001&radius=40000&categories=sailing&sort_by=rating
processing Record 55 of Set 2 | https://api.yelp.com/v3/businesses/search?latitude=34.244058700000004&longitude=-116.91018000000001&radius=40000&categories=snorkeling&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -116.

...

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 47.9734174, 'latitude': 29.3796532}}}
processing Record 82 of Set 45 | https://api.yelp.com/v3/businesses/search?latitude=29.3796532&longitude=47.9734174&radius=40000&categories=ziplining&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 47.9734174, 'latitude': 29.3796532}}}
processing Record 83 of Set 45 | https://api.yelp.com/v3/businesses/search?latitude=29.3796532&longitude=47.9734174&radius=40000&categories=spas&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 47.9734174, 'latitude': 29.3796532}}}
processing Record 84 of Set 45 | https://api.yelp.com/v3/businesses/search?latitude=29.3796532&longitude=47.9734174&radius=40000&categories=hotsprings&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': 

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 35.478429999999996, 'latitude': 33.8959203}}}
processing Record 16 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=33.8959203&longitude=35.478429999999996&radius=40000&categories=hanggliding&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 35.478429999999996, 'latitude': 33.8959203}}}
processing Record 17 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=33.8959203&longitude=35.478429999999996&radius=40000&categories=horsebackriding&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 35.478429999999996, 'latitude': 33.8959203}}}
processing Record 18 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=33.8959203&longitude=35.478429999999996&radius=40000&categories=hiking&sort_by=rating
********URL error: Skip

processing Record 44 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=47.3744489&longitude=8.5410422&radius=40000&categories=snorkeling&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 8.5410422, 'latitude': 47.3744489}}}
processing Record 45 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=47.3744489&longitude=8.5410422&radius=40000&categories=ziplining&sort_by=rating
processing Record 46 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=47.3744489&longitude=8.5410422&radius=40000&categories=spas&sort_by=rating
processing Record 47 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=47.3744489&longitude=8.5410422&radius=40000&categories=hotsprings&sort_by=rating
processing Record 48 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=47.3744489&longitude=8.5410422&radius=40000&categories=massage&sort_by=rating
processing Record 49 of Set 46 | https://ap

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 14.513675899999999, 'latitude': 35.898981799999994}}}
processing Record 84 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=35.898981799999994&longitude=14.513675899999999&radius=40000&categories=parasailing&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 14.513675899999999, 'latitude': 35.898981799999994}}}
processing Record 85 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=35.898981799999994&longitude=14.513675899999999&radius=40000&categories=sailing&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 14.513675899999999, 'latitude': 35.898981799999994}}}
processing Record 86 of Set 46 | https://api.yelp.com/v3/businesses/search?latitude=35.898981799999994&longitude=14.513675899999999&radius=40000&categories=snork

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -21.9422367, 'latitude': 64.145981}}}
processing Record 11 of Set 47 | https://api.yelp.com/v3/businesses/search?latitude=64.145981&longitude=-21.9422367&radius=40000&categories=massage&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -21.9422367, 'latitude': 64.145981}}}
processing Record 12 of Set 47 | https://api.yelp.com/v3/businesses/search?latitude=64.145981&longitude=-21.9422367&radius=40000&categories=bedbreakfast&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -21.9422367, 'latitude': 64.145981}}}
processing Record 13 of Set 47 | https://api.yelp.com/v3/businesses/search?latitude=64.145981&longitude=-21.9422367&radius=40000&categories=hotels&sort_by=rating
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region'

In [51]:
# convert the list of dictionaries to a dataframe
yelp_df = pd.DataFrame(data_list)   
yelp_df.head()

,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,38.046407,-84.497039,beaches,1,4.500,2.000000
1,38.046407,-84.497039,horsebackriding,20,3.600,10.600000
2,38.046407,-84.497039,hiking,9,4.000,7.444444
3,38.046407,-84.497039,beaches,1,4.500,2.000000
4,38.046407,-84.497039,horsebackriding,20,3.600,10.600000
...,...,...,...,...,...,...
2315,46.201756,6.146601,restaurants,20,5.000,6.900000
2316,46.201756,6.146601,nightlife,20,4.675,7.850000
2317,35.898982,14.513676,ziplining,1,4.500,3.000000
2318,35.898982,14.513676,transportation,1,4.500,3.000000


In [25]:
# Convert the list of errors into a dataframe to view any similarities
errors_df = pd.DataFrame(url_errors)
errors_df.head()

,url,latitude,longitude
0,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039
1,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039
2,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039
3,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039
4,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039
...,...,...,...
2298,https://api.yelp.com/v3/businesses/search?lati...,64.145981,-21.942237
2299,https://api.yelp.com/v3/businesses/search?lati...,64.145981,-21.942237
2300,https://api.yelp.com/v3/businesses/search?lati...,64.145981,-21.942237
2301,https://api.yelp.com/v3/businesses/search?lati...,64.145981,-21.942237


In [26]:
# determine if there are any missing reviews. All 21 categories are accounted for
yelp_df["category"].value_counts()

transportation      161
ziplining           161
restaurants         157
hotels              148
nightlife           148
tours               144
massage             143
spas                143
bedbreakfast        139
horsebackriding     137
hiking              133
beaches             131
resorts             125
sailing             105
paddleboarding       90
hot_air_balloons     77
skiresorts           74
snorkeling           46
hanggliding          24
parasailing          19
hotsprings           15
Name: category, dtype: int64

In [ ]:
# Check how many categories successfully returned for each of the locations that returned.
# We are missing many locations, as the length should be 237. None of our locations have the full 21 activities either.
yelp_df["latitude"].nunique()

In [94]:
# Separate out all of the values that have less than 5 actual activities returned successfully
activity_df = pd.DataFrame(yelp_df.loc[(yelp_df["category"] != "transportation") & (yelp_df["category"] !="restaurants") &
                                       (yelp_df["category"] != "hotels")         & (yelp_df["category"] != "nightlife")])
value_counts = pd.DataFrame(activity_df[["latitude","longitude"]].groupby(["latitude","longitude"]).size().sort_values(ascending=False))
value_counts = value_counts.rename(columns={0:"activityCount","longitude":"test"})
value_counts = value_counts.loc[(value_counts["activityCount"] < 5)]
value_counts = value_counts.reset_index(inplace=False)
value_counts.head()

18

In [93]:
# Filter the list of results to only include locations with five or more actual activities
longitudes_list = value_counts["longitude"].tolist()
check_five = yelp_df[~yelp_df["longitude"].isin(longitudes_list)]
check_five

,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,38.046407,-84.497039,beaches,1,4.500,2.000000
1,38.046407,-84.497039,horsebackriding,20,3.600,10.600000
2,38.046407,-84.497039,hiking,9,4.000,7.444444
3,38.046407,-84.497039,beaches,1,4.500,2.000000
4,38.046407,-84.497039,horsebackriding,20,3.600,10.600000
...,...,...,...,...,...,...
2312,46.201756,6.146601,skiresorts,20,3.150,1.500000
2313,46.201756,6.146601,tours,20,3.050,2.250000
2314,46.201756,6.146601,transportation,20,5.000,7.850000
2315,46.201756,6.146601,restaurants,20,5.000,6.900000


In [96]:
# Create a dataframe of all null values in the yelp dataframe
null_df = pd.merge(check_five,locations_df,how="right",on=["latitude","longitude"])
null_df = null_df[null_df["category"].isna()]
null_df

,latitude,longitude,category,results_total,avg_rating,avg_review_count,Unnamed: 0,City,State/Province,Country
2256,-19.996972,-157.785871,NaN,NaN,NaN,NaN,2,Cook Islands​,NaN,Cook Islands​
2257,42.648736,18.094680,NaN,NaN,NaN,NaN,10,Dubrovnik,Croatia,Croatia
2258,17.223472,-61.955461,NaN,NaN,NaN,NaN,13,Antigua and Barbuda​,NaN,Antigua and Barbuda​
2259,25.265347,55.292491,NaN,NaN,NaN,NaN,14,Dubai,United Arab Emirates,United Arab Emirates
2260,21.721746,-71.552781,NaN,NaN,NaN,NaN,16,Turks and Caicos Islands​,NaN,Turks and Caicos Islands​
...,...,...,...,...,...,...,...,...,...,...
2325,23.588202,58.382945,NaN,NaN,NaN,NaN,226,Muscat,NaN,Oman
2326,29.379653,47.973417,NaN,NaN,NaN,NaN,230,Kuwait City,NaN,Kuwait
2327,33.895920,35.478430,NaN,NaN,NaN,NaN,232,Beirut,NaN,Lebanon
2328,35.898982,14.513676,NaN,NaN,NaN,NaN,235,Valletta,NaN,Malta


In [97]:
# Export list of all locations with 5+ activities
yelp_df.to_csv("../Resources/yelpdata.csv")

In [98]:
# Export list of all locations with 0-4 activities
null_df.to_csv("../Resources/missingActivities.csv")